<a href="https://colab.research.google.com/github/pedro-pauletti/metaheurisct-algorithms-human-based-study/blob/main/Estudo_Algoritmos_Otimiza%C3%A7%C3%A3o_Baseados_em_Humanos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Requirements

In [ ]:
!pip install mealpy==3.0.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.3/386.3 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 21.5 MB/s eta 0:00:00


### Libraries

In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris, load_diabetes, load_wine, load_breast_cancer, load_digits
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn import datasets, metrics
from sklearn import metrics
from mealpy import FloatVar, StringVar, IntegerVar, BoolVar, MixedSetVar, Problem
import json
import os

In [ ]:
from mealpy import BBO, PSO, GA, ALO, AO, ARO, AVOA, BA, BBOA, BMO, EOA, IWO
from mealpy import SBO, SMA, SOA, SOS, TPO, TSA, VCS, WHO, AOA, CEM, CGO, CircleSA, GBO, HC, INFO, PSS, RUN, SCA
from mealpy import SHIO, TS, HS, AEO, GCO, WCA, CRO, DE, EP, ES, FPA, MA, SHADE, BRO, BSO, CA, CHIO, FBIO, GSKA, HBO
from mealpy import HCO, ICA, LCO, WarSO, TOA, TLO, SSDO, SPBO, SARO, QSA, ArchOA, ASO, CDO, EFO, EO, EVO, FLA
from mealpy import HGSO, MVO, NRO, RIME, SA, WDO, TWO, ABC, ACOR, AGTO, BeesA, BES, BFO, ZOA, WOA, WaOA, TSO
from mealpy import TDO, STO, SSpiderO, SSpiderA, SSO, SSA, SRSR, SLO, SHO, SFO, ServalOA, SeaHO, SCSO, POA
from mealpy import PFA, OOA, NGO, NMRA, MSA, MRFO, MPA, MGO, MFO, JA, HHO, HGS, HBA, GWO, GTO, GOA
from mealpy import GJO, FOX, FOA, FFO, FFA, FA, ESOA, EHO, DO, DMOA, CSO, CSA, CoatiOA, COA, BSA
from mealpy import StringVar, FloatVar, BoolVar, PermutationVar, MixedSetVar, IntegerVar, BinaryVar
from mealpy import Tuner, Multitask, Problem, Optimizer, Termination, ParameterGrid
from mealpy import get_all_optimizers, get_optimizer_by_name

### Defining Optimization Algorithms

In [ ]:
epoch = 50
pop_size = 50

paras_bro = {
    "epoch": epoch,
    "pop_size": pop_size,
    "threshold": 3,
}
paras_improved_bso = {
    "epoch": epoch,
    "pop_size": pop_size,
    "m_clusters": 5,
    "p1": 0.2,
    "p2": 0.8,
    "p3": 0.4,
    "p4": 0.5,
}
paras_bso = dict(paras_improved_bso)
paras_bso["slope"] = 20
paras_ca = {
    "epoch": epoch,
    "pop_size": pop_size,
    "accepted_rate": 0.15,
}
paras_chio = {
    "epoch": epoch,
    "pop_size": pop_size,
    "brr": 0.15,
    "max_age": 3
}
paras_fbio = {
    "epoch": epoch,
    "pop_size": pop_size,
}
paras_gska = {
    "epoch": epoch,
    "pop_size": pop_size,
    "pb": 0.1,
    "kf": 0.5,
    "kr": 0.9,
    "kg": 5,
}
paras_ica = {
    "epoch": epoch,
    "pop_size": pop_size,
    "empire_count": 5,
    "assimilation_coeff": 1.5,
    "revolution_prob": 0.05,
    "revolution_rate": 0.1,
    "revolution_step_size": 0.1,
    "zeta": 0.1,
}
paras_lco = {
    "epoch": epoch,
    "pop_size": pop_size,
    "r1": 2.35,
}

paras_qsa = {
    "epoch": epoch,
    "pop_size": pop_size,
}
paras_saro = {
    "epoch": epoch,
    "pop_size": pop_size,
    "se": 0.5,
    "mu": 15
}
paras_ssdo = {
    "epoch": epoch,
    "pop_size": pop_size,
}
paras_tlo = {
    "epoch": epoch,
    "pop_size": pop_size,
}

term_dict = {
   "max_early_stop": 20
}

In [ ]:
# Lista de algoritmos de otimização da mealpy
algorithms = {
    "BRO": BRO.OriginalBRO(**paras_bro),
    "BSO": BSO.OriginalBSO(**paras_bso),
    "CA": CA.OriginalCA(**paras_ca),
    "CHIO": CHIO.OriginalCHIO(**paras_chio),
    "FBIO": FBIO.OriginalFBIO(**paras_fbio),
    "GSKA": GSKA.OriginalGSKA(**paras_gska),
    "ICA": ICA.OriginalICA(**paras_ica),
    "LCO": LCO.OriginalLCO(**paras_lco),
    "QSA": QSA.OriginalQSA(**paras_qsa),
    "SARO": SARO.OriginalSARO(**paras_saro),
    "SSDO": SSDO.OriginalSSDO(**paras_ssdo),
    "TLO": TLO.OriginalTLO(**paras_tlo)
}

In [ ]:
# Função para salvar listas em CSV
def save_history_to_csv(model, prefix):
    # Define um dicionário para armazenar os dados
    history_data = {
        "list_global_best_fit": model.history.list_global_best_fit,
        "list_exploitation": model.history.list_exploitation,
        "list_exploration": model.history.list_exploration,
        "list_population": model.history.list_population,
        "list_global_best": model.history.list_global_best,
        "list_current_best": model.history.list_current_best,
        "list_current_best_fit": model.history.list_current_best_fit,
        "list_diversity": model.history.list_diversity,
        "list_current_worst": model.history.list_current_worst,
        "list_global_worst": model.history.list_global_worst,
        "list_epoch_time": model.history.list_epoch_time,
    }

    # Salva cada lista em um arquivo CSV separado
    for key, value in history_data.items():
        df = pd.DataFrame({key: value})
        df.to_csv(f"{prefix}/{key}.csv", index=False)

### Optimize ML Model

In [ ]:
# Load the data set; In this example, the breast cancer dataset is loaded.
X, y = datasets.load_breast_cancer(return_X_y=True)

# Create training and test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1, stratify=y)

sc = StandardScaler()
X_train_std = sc.fit_transform(X_train)
X_test_std = sc.transform(X_test)

data = {
    "X_train": X_train_std,
    "X_test": X_test_std,
    "y_train": y_train,
    "y_test": y_test
}

In [ ]:
class SvmOptimizedProblem(Problem):
    def __init__(self, bounds=None, minmax="max", data=None, **kwargs):
        self.data = data
        super().__init__(bounds, minmax, **kwargs)

    def obj_func(self, x):
        x_decoded = self.decode_solution(x)
        C_paras, kernel_paras = x_decoded["C_paras"], x_decoded["kernel_paras"]
        degree, gamma, probability = x_decoded["degree_paras"], x_decoded["gamma_paras"], x_decoded["probability_paras"]

        svc = SVC(C=C_paras, kernel=kernel_paras, degree=degree,
                  gamma=gamma, probability=probability, random_state=1)
        # Fit the model
        svc.fit(self.data["X_train"], self.data["y_train"])
        # Make the predictions
        y_predict = svc.predict(self.data["X_test"])
        # Measure the performance
        return metrics.accuracy_score(self.data["y_test"], y_predict)

my_bounds = [
    FloatVar(lb=0.01, ub=1000., name="C_paras"),
    StringVar(valid_sets=('linear', 'poly', 'rbf', 'sigmoid'), name="kernel_paras"),
    IntegerVar(lb=1, ub=5, name="degree_paras"),
    MixedSetVar(valid_sets=('scale', 'auto', 0.01, 0.05, 0.1, 0.5, 1.0), name="gamma_paras"),
    BoolVar(n_vars=1, name="probability_paras"),
]

In [ ]:
for algo_name, algo in algorithms.items():
      print(f"\n\n==== Algorithm: {algo_name} =====")
      problem = SvmOptimizedProblem(bounds=my_bounds, minmax="max", data=data)
      model = algo
      model.solve(problem, termination=term_dict)
      print(f"Best agent: {model.g_best}")
      print(f"Best solution: {model.g_best.solution}")
      print(f"Best accuracy: {model.g_best.target.fitness}")
      print(f"Best parameters: {model.problem.decode_solution(model.g_best.solution)}")
      model.history.save_global_objectives_chart(filename=f"{algo_name}/optimize-ml-model/goc")
      model.history.save_local_objectives_chart(filename=f"{algo_name}/optimize-ml-model/loc")
      model.history.save_global_best_fitness_chart(filename=f"{algo_name}/optimize-ml-model/gbfc")
      model.history.save_local_best_fitness_chart(filename=f"{algo_name}/optimize-ml-model/lbfc")
      model.history.save_runtime_chart(filename=f"{algo_name}/optimize-ml-model/rtc")
      model.history.save_exploration_exploitation_chart(filename=f"{algo_name}/optimize-ml-model/eec")
      model.history.save_diversity_chart(filename=f"{algo_name}/optimize-ml-model/dc")
      save_history_to_csv(model, f"{algo_name}/optimize-ml-model")


INFO:mealpy.human_based.BRO.OriginalBRO:Solving single objective optimization problem.




==== Algorithm: BRO =====


INFO:mealpy.human_based.BRO.OriginalBRO:>>>Problem: P, Epoch: 1, Current best: 0.9766081871345029, Global best: 0.9766081871345029, Runtime: 5.61558 seconds
INFO:mealpy.human_based.BRO.OriginalBRO:>>>Problem: P, Epoch: 2, Current best: 0.9766081871345029, Global best: 0.9766081871345029, Runtime: 5.49490 seconds
INFO:mealpy.human_based.BRO.OriginalBRO:>>>Problem: P, Epoch: 3, Current best: 0.9766081871345029, Global best: 0.9766081871345029, Runtime: 6.08654 seconds
INFO:mealpy.human_based.BRO.OriginalBRO:>>>Problem: P, Epoch: 4, Current best: 0.9766081871345029, Global best: 0.9766081871345029, Runtime: 5.27403 seconds
INFO:mealpy.human_based.BRO.OriginalBRO:>>>Problem: P, Epoch: 5, Current best: 0.9766081871345029, Global best: 0.9766081871345029, Runtime: 3.12941 seconds
INFO:mealpy.human_based.BRO.OriginalBRO:>>>Problem: P, Epoch: 6, Current best: 0.9766081871345029, Global best: 0.9766081871345029, Runtime: 3.13703 seconds
INFO:mealpy.human_based.BRO.OriginalBRO:>>>Problem: P, Epo

Best agent: id: 1107, target: Objectives: [0.97660819], Fitness: 0.9766081871345029, solution: [195.18934698   2.           3.           2.           1.16724824]
Best solution: [195.18934698   2.           3.           2.           1.16724824]
Best accuracy: 0.9766081871345029
Best parameters: {'C_paras': 195.18934698020306, 'kernel_paras': 'rbf', 'degree_paras': 3, 'gamma_paras': 0.01, 'probability_paras': True}


INFO:mealpy.human_based.BSO.OriginalBSO:Solving single objective optimization problem.


...

Best agent: id: 47201, target: Objectives: [0.97660819], Fitness: 0.9766081871345029, solution: [8.17098531 2.         0.5        0.         0.54706455]
Best solution: [8.17098531 2.         0.5        0.         0.54706455]
Best accuracy: 0.9766081871345029
Best parameters: {'C_paras': 8.170985309362024, 'kernel_paras': 'rbf', 'degree_paras': 1, 'gamma_paras': 'auto', 'probability_paras': False}


INFO:mealpy.human_based.TLO.OriginalTLO:Solving single objective optimization problem.




==== Algorithm: TLO =====


INFO:mealpy.human_based.TLO.OriginalTLO:>>>Problem: P, Epoch: 1, Current best: 0.9766081871345029, Global best: 0.9766081871345029, Runtime: 2.16434 seconds
INFO:mealpy.human_based.TLO.OriginalTLO:>>>Problem: P, Epoch: 2, Current best: 0.9766081871345029, Global best: 0.9766081871345029, Runtime: 1.88234 seconds
INFO:mealpy.human_based.TLO.OriginalTLO:>>>Problem: P, Epoch: 3, Current best: 0.9766081871345029, Global best: 0.9766081871345029, Runtime: 1.23599 seconds
INFO:mealpy.human_based.TLO.OriginalTLO:>>>Problem: P, Epoch: 4, Current best: 0.9766081871345029, Global best: 0.9766081871345029, Runtime: 1.23669 seconds
INFO:mealpy.human_based.TLO.OriginalTLO:>>>Problem: P, Epoch: 5, Current best: 0.9766081871345029, Global best: 0.9766081871345029, Runtime: 1.23896 seconds
INFO:mealpy.human_based.TLO.OriginalTLO:>>>Problem: P, Epoch: 6, Current best: 0.9766081871345029, Global best: 0.9766081871345029, Runtime: 1.06996 seconds
INFO:mealpy.human_based.TLO.OriginalTLO:>>>Problem: P, Epo

Best agent: id: 50215, target: Objectives: [0.98245614], Fitness: 0.9824561403508771, solution: [40.23684384  2.          5.4999      2.          0.        ]
Best solution: [40.23684384  2.          5.4999      2.          0.        ]
Best accuracy: 0.9824561403508771
Best parameters: {'C_paras': 40.23684384000167, 'kernel_paras': 'rbf', 'degree_paras': 5, 'gamma_paras': 0.01, 'probability_paras': False}


In [ ]:
# Funções e algoritmos
algorithms = [ "BRO", "BSO", "CA", "CHIO", "FBIO", "GSKA", "ICA", "LCO", "QSA", "SARO", "SSDO", "TLO"]
metrics = ["list_global_best_fit", "list_exploitation", "list_exploration", "list_current_best_fit", "list_diversity", "list_epoch_time"]

# Caminho base dos dados
base_path = ""  # Substitua pelo caminho real dos dados

# Função para carregar os dados e consolidar em um DataFrame
def consolidate_data(metric):
    consolidated_df = pd.DataFrame()
    for alg in algorithms:
        file_path = os.path.join(base_path, f"{alg}/optimize-ml-model/{metric}.csv")
        if os.path.exists(file_path):
            df = pd.read_csv(file_path)
            df.columns = [f"{alg}_{col}" for col in df.columns]
            if consolidated_df.empty:
                consolidated_df = df
            else:
                consolidated_df = pd.concat([consolidated_df, df], axis=1)
        else:
            print(f"Arquivo não encontrado: {file_path}")
    return consolidated_df

# Processar cada métrica
for metric in metrics:
    consolidated_df = consolidate_data(metric)
    output_file = f"results/optimize-ml-model/{metric}.csv"
    consolidated_df.to_csv(output_file, index=False)
    print(f"Arquivo salvo: {output_file}")


Arquivo salvo: results/optimize-ml-model/list_global_best_fit.csv
Arquivo salvo: results/optimize-ml-model/list_exploitation.csv
Arquivo salvo: results/optimize-ml-model/list_exploration.csv
Arquivo salvo: results/optimize-ml-model/list_current_best_fit.csv
Arquivo salvo: results/optimize-ml-model/list_diversity.csv
Arquivo salvo: results/optimize-ml-model/list_epoch_time.csv


### Optimize Funcitions

In [ ]:
epoch = 10000
pop_size = 50

paras_bro = {
    "epoch": epoch,
    "pop_size": pop_size,
    "threshold": 3,
}
paras_improved_bso = {
    "epoch": epoch,
    "pop_size": pop_size,
    "m_clusters": 5,
    "p1": 0.2,
    "p2": 0.8,
    "p3": 0.4,
    "p4": 0.5,
}
paras_bso = dict(paras_improved_bso)
paras_bso["slope"] = 20
paras_ca = {
    "epoch": epoch,
    "pop_size": pop_size,
    "accepted_rate": 0.15,
}
paras_chio = {
    "epoch": epoch,
    "pop_size": pop_size,
    "brr": 0.15,
    "max_age": 3
}
paras_fbio = {
    "epoch": epoch,
    "pop_size": pop_size,
}
paras_gska = {
    "epoch": epoch,
    "pop_size": pop_size,
    "pb": 0.1,
    "kf": 0.5,
    "kr": 0.9,
    "kg": 5,
}
paras_ica = {
    "epoch": epoch,
    "pop_size": pop_size,
    "empire_count": 5,
    "assimilation_coeff": 1.5,
    "revolution_prob": 0.05,
    "revolution_rate": 0.1,
    "revolution_step_size": 0.1,
    "zeta": 0.1,
}
paras_lco = {
    "epoch": epoch,
    "pop_size": pop_size,
    "r1": 2.35,
}

paras_qsa = {
    "epoch": epoch,
    "pop_size": pop_size,
}
paras_saro = {
    "epoch": epoch,
    "pop_size": pop_size,
    "se": 0.5,
    "mu": 15
}
paras_ssdo = {
    "epoch": epoch,
    "pop_size": pop_size,
}
paras_tlo = {
    "epoch": epoch,
    "pop_size": pop_size,
}

term_dict = {
    "max_fe": 3000
}

In [ ]:
# Lista de algoritmos de otimização da mealpy
algorithms = {
    "BRO": BRO.OriginalBRO(**paras_bro),
    "BSO": BSO.OriginalBSO(**paras_bso),
    "CA": CA.OriginalCA(**paras_ca),
    "CHIO": CHIO.OriginalCHIO(**paras_chio),
    "FBIO": FBIO.OriginalFBIO(**paras_fbio),
    "GSKA": GSKA.OriginalGSKA(**paras_gska),
    # "ICA": ICA.OriginalICA(**paras_ica),
    "LCO": LCO.OriginalLCO(**paras_lco),
    "QSA": QSA.OriginalQSA(**paras_qsa),
    "SARO": SARO.OriginalSARO(**paras_saro),
    "SSDO": SSDO.OriginalSSDO(**paras_ssdo),
    "TLO": TLO.OriginalTLO(**paras_tlo)
}

In [ ]:
## Import libraries
from opfunu.cec_based.cec2022 import F12022, F22022, F32022, F42022, F52022, F62022, F72022, F82022, F92022, F102022, F112022

problem_size = 20
f_bias = 0

functions = {
    "F12022": F12022(problem_size, f_bias=f_bias),
    "F22022": F22022(problem_size, f_bias=f_bias),
    "F32022": F32022(problem_size, f_bias=f_bias),
    "F42022": F42022(problem_size, f_bias=f_bias),
    "F52022": F52022(problem_size, f_bias=f_bias),
    "F62022": F62022(problem_size, f_bias=f_bias),
    "F72022": F72022(problem_size, f_bias=f_bias),
    "F82022": F82022(problem_size, f_bias=f_bias),
    "F92022": F92022(problem_size, f_bias=f_bias),
    "F102022": F102022(problem_size, f_bias=f_bias),
    "F112022": F112022(problem_size, f_bias=f_bias),
}

for algo_name, algo in algorithms.items():
  for func_name, func in functions.items():

      print(f"\n\n==== Function: {func_name} =====")
      f = func

      problem_dict = {
          "obj_func": f.evaluate,
          "bounds": FloatVar(lb=f.lb, ub=f.ub),
          "minmax": "min",
          "name": func_name,
          "log_to": "console",
      }

      model = algo
      model.solve(problem_dict, termination=term_dict)
      print(f"Best agent: {model.g_best}")
      print(f"Best solution: {model.g_best.solution}")
      print(f"Best accuracy: {model.g_best.target.fitness}")
      print(f"Best parameters: {model.problem.decode_solution(model.g_best.solution)}")
      model.history.save_global_objectives_chart(filename=f"{algo_name}/optimize-function/{func_name}/goc")
      model.history.save_local_objectives_chart(filename=f"{algo_name}/optimize-function/{func_name}/loc")
      model.history.save_global_best_fitness_chart(filename=f"{algo_name}/optimize-function/{func_name}/gbfc")
      model.history.save_local_best_fitness_chart(filename=f"{algo_name}/optimize-function/{func_name}/lbfc")
      model.history.save_runtime_chart(filename=f"{algo_name}/optimize-function/{func_name}/rtc")
      model.history.save_exploration_exploitation_chart(filename=f"{algo_name}/optimize-function/{func_name}/eec")
      model.history.save_diversity_chart(filename=f"{algo_name}/optimize-function/{func_name}/dc")
      save_history_to_csv(model, f"{algo_name}/optimize-function/{func_name}")

INFO:mealpy.human_based.BRO.OriginalBRO:Solving single objective optimization problem.
INFO:mealpy.human_based.BRO.OriginalBRO:>>>Problem: F12022, Epoch: 1, Current best: 77224.89390471639, Global best: 77224.89390471639, Runtime: 0.00864 seconds
INFO:mealpy.human_based.BRO.OriginalBRO:>>>Problem: F12022, Epoch: 2, Current best: 77224.89390471639, Global best: 77224.89390471639, Runtime: 0.01075 seconds
INFO:mealpy.human_based.BRO.OriginalBRO:>>>Problem: F12022, Epoch: 3, Current best: 77224.89390471639, Global best: 77224.89390471639, Runtime: 0.01206 seconds
INFO:mealpy.human_based.BRO.OriginalBRO:>>>Problem: F12022, Epoch: 4, Current best: 77224.89390471639, Global best: 77224.89390471639, Runtime: 0.01174 seconds
INFO:mealpy.human_based.BRO.OriginalBRO:>>>Problem: F12022, Epoch: 5, Current best: 77224.89390471639, Global best: 77224.89390471639, Runtime: 0.01248 seconds
INFO:mealpy.human_based.BRO.OriginalBRO:>>>Problem: F12022, Epoch: 6, Current best: 77224.89390471639, Global bes



==== Function: F12022 =====


INFO:mealpy.human_based.BRO.OriginalBRO:>>>Problem: F12022, Epoch: 15, Current best: 77224.89390471639, Global best: 77224.89390471639, Runtime: 0.01501 seconds
INFO:mealpy.human_based.BRO.OriginalBRO:>>>Problem: F12022, Epoch: 16, Current best: 77224.89390471639, Global best: 77224.89390471639, Runtime: 0.01172 seconds
INFO:mealpy.human_based.BRO.OriginalBRO:>>>Problem: F12022, Epoch: 17, Current best: 77224.89390471639, Global best: 77224.89390471639, Runtime: 0.01156 seconds
INFO:mealpy.human_based.BRO.OriginalBRO:>>>Problem: F12022, Epoch: 18, Current best: 77224.89390471639, Global best: 77224.89390471639, Runtime: 0.01143 seconds
INFO:mealpy.human_based.BRO.OriginalBRO:>>>Problem: F12022, Epoch: 19, Current best: 33788.95101783039, Global best: 33788.95101783039, Runtime: 0.01161 seconds
INFO:mealpy.human_based.BRO.OriginalBRO:>>>Problem: F12022, Epoch: 20, Current best: 33788.95101783039, Global best: 33788.95101783039, Runtime: 0.01102 seconds
INFO:mealpy.human_based.BRO.Origin

Best agent: id: 6258, target: Objectives: [33788.95101783], Fitness: 33788.95101783039, solution: [-60.9517185  -60.27147563 -62.21265454 -45.37033358 -21.85227123
  36.95159486  -7.68272626 -25.61322623 -72.45909429 -30.67963329
 -41.91697212 -13.58987399  11.62042315 -36.61820526  19.67453161
  10.30640111  71.06520762  42.17498686  -6.38079246 -74.31186468]
Best solution: [-60.9517185  -60.27147563 -62.21265454 -45.37033358 -21.85227123
  36.95159486  -7.68272626 -25.61322623 -72.45909429 -30.67963329
 -41.91697212 -13.58987399  11.62042315 -36.61820526  19.67453161
  10.30640111  71.06520762  42.17498686  -6.38079246 -74.31186468]
Best accuracy: 33788.95101783039
Best parameters: {'float': array([-60.9517185 , -60.27147563, -62.21265454, -45.37033358,
       -21.85227123,  36.95159486,  -7.68272626, -25.61322623,
       -72.45909429, -30.67963329, -41.91697212, -13.58987399,
        11.62042315, -36.61820526,  19.67453161,  10.30640111,
        71.06520762,  42.17498686,  -6.380792

INFO:mealpy.human_based.BRO.OriginalBRO:Solving single objective optimization problem.
INFO:mealpy.human_based.BRO.OriginalBRO:>>>Problem: F22022, Epoch: 1, Current best: 7616.3768404851535, Global best: 7616.3768404851535, Runtime: 0.00839 seconds
INFO:mealpy.human_based.BRO.OriginalBRO:>>>Problem: F22022, Epoch: 2, Current best: 7616.3768404851535, Global best: 7616.3768404851535, Runtime: 0.01429 seconds
INFO:mealpy.human_based.BRO.OriginalBRO:>>>Problem: F22022, Epoch: 3, Current best: 7616.3768404851535, Global best: 7616.3768404851535, Runtime: 0.01166 seconds
INFO:mealpy.human_based.BRO.OriginalBRO:>>>Problem: F22022, Epoch: 4, Current best: 7616.3768404851535, Global best: 7616.3768404851535, Runtime: 0.01117 seconds
INFO:mealpy.human_based.BRO.OriginalBRO:>>>Problem: F22022, Epoch: 5, Current best: 5541.169907180378, Global best: 5541.169907180378, Runtime: 0.01125 seconds
INFO:mealpy.human_based.BRO.OriginalBRO:>>>Problem: F22022, Epoch: 6, Current best: 5541.169907180378, Gl

...

Best agent: id: 685655, target: Objectives: [443.75975712], Fitness: 443.759757123892, solution: [-33.24316067 -35.530387   -54.621896   -48.05675959  -8.58447625
 -27.15769077  36.30121802  -4.13922423  51.16950007  -8.87200798
  27.83191632  22.80149891  39.08228282  31.27307176  48.84981665
  22.12923629 -16.11110199  19.66762055  10.83423197  64.45468036]
Best solution: [-33.24316067 -35.530387   -54.621896   -48.05675959  -8.58447625
 -27.15769077  36.30121802  -4.13922423  51.16950007  -8.87200798
  27.83191632  22.80149891  39.08228282  31.27307176  48.84981665
  22.12923629 -16.11110199  19.66762055  10.83423197  64.45468036]
Best accuracy: 443.759757123892
Best parameters: {'float': array([-33.24316067, -35.530387  , -54.621896  , -48.05675959,
        -8.58447625, -27.15769077,  36.30121802,  -4.13922423,
        51.16950007,  -8.87200798,  27.83191632,  22.80149891,
        39.08228282,  31.27307176,  48.84981665,  22.12923629,
       -16.11110199,  19.66762055,  10.83423197

INFO:mealpy.human_based.TLO.OriginalTLO:Solving single objective optimization problem.
INFO:mealpy.human_based.TLO.OriginalTLO:>>>Problem: F112022, Epoch: 1, Current best: 15797.315351302352, Global best: 15797.315351302352, Runtime: 0.07933 seconds




==== Function: F112022 =====


INFO:mealpy.human_based.TLO.OriginalTLO:>>>Problem: F112022, Epoch: 2, Current best: 6554.3237840290485, Global best: 6554.3237840290485, Runtime: 0.09646 seconds
INFO:mealpy.human_based.TLO.OriginalTLO:>>>Problem: F112022, Epoch: 3, Current best: 5904.2856010316655, Global best: 5904.2856010316655, Runtime: 0.08724 seconds
INFO:mealpy.human_based.TLO.OriginalTLO:>>>Problem: F112022, Epoch: 4, Current best: 4034.6034335972486, Global best: 4034.6034335972486, Runtime: 0.08420 seconds
INFO:mealpy.human_based.TLO.OriginalTLO:>>>Problem: F112022, Epoch: 5, Current best: 4034.6034335972486, Global best: 4034.6034335972486, Runtime: 0.08759 seconds
INFO:mealpy.human_based.TLO.OriginalTLO:>>>Problem: F112022, Epoch: 6, Current best: 2959.6714456002983, Global best: 2959.6714456002983, Runtime: 0.07853 seconds
INFO:mealpy.human_based.TLO.OriginalTLO:>>>Problem: F112022, Epoch: 7, Current best: 1594.4035404521758, Global best: 1594.4035404521758, Runtime: 0.08685 seconds
INFO:mealpy.human_base

Best agent: id: 688771, target: Objectives: [32.97847363], Fitness: 32.97847362753611, solution: [ 46.26420119   4.79373949 -39.28116133  10.33044719  60.33587022
  53.99693355  25.44091947  33.9151479   22.39930054 -47.55169262
  57.34774562 -59.1400447   63.9940206   -1.34412621 -55.64589883
  13.91817591  13.04703127 -61.8886846   59.4401284  -52.93799575]
Best solution: [ 46.26420119   4.79373949 -39.28116133  10.33044719  60.33587022
  53.99693355  25.44091947  33.9151479   22.39930054 -47.55169262
  57.34774562 -59.1400447   63.9940206   -1.34412621 -55.64589883
  13.91817591  13.04703127 -61.8886846   59.4401284  -52.93799575]
Best accuracy: 32.97847362753611
Best parameters: {'float': array([ 46.26420119,   4.79373949, -39.28116133,  10.33044719,
        60.33587022,  53.99693355,  25.44091947,  33.9151479 ,
        22.39930054, -47.55169262,  57.34774562, -59.1400447 ,
        63.9940206 ,  -1.34412621, -55.64589883,  13.91817591,
        13.04703127, -61.8886846 ,  59.4401284

In [ ]:
import pandas as pd
import os

# Funções e algoritmos
functions = ["F12022", "F22022", "F32022", "F42022", "F52022", "F62022", "F72022", "F82022", "F92022", "F102022", "F112022"]
algorithms = [ "BRO", "BSO", "CA", "CHIO", "FBIO", "GSKA", "ICA", "LCO", "QSA", "SARO", "SSDO", "TLO"]
metrics = ["list_global_best_fit", "list_exploitation", "list_exploration", "list_current_best_fit", "list_diversity", "list_epoch_time"]

# Caminho base dos dados
base_path = ""  # Substitua pelo caminho real dos dados

# Função para carregar os dados e consolidar em um DataFrame
def consolidate_data(func_name, metric):
    consolidated_df = pd.DataFrame()
    for alg in algorithms:
        file_path = os.path.join(base_path, alg, f"optimize-function/{func_name}/{metric}.csv")
        if os.path.exists(file_path):
            df = pd.read_csv(file_path)
            df.columns = [f"{alg}_{col}" for col in df.columns]
            if consolidated_df.empty:
                consolidated_df = df
            else:
                consolidated_df = pd.concat([consolidated_df, df], axis=1)
        else:
            print(f"Arquivo não encontrado: {file_path}")
    return consolidated_df

# Processar cada função e métrica
for func_name in functions:
    for metric in metrics:
        consolidated_df = consolidate_data(func_name, metric)
        output_file = f"results/optimize-function/{func_name}_{metric}.csv"
        consolidated_df.to_csv(output_file, index=False)
        print(f"Arquivo salvo: {output_file}")


Arquivo não encontrado: ICA/optimize-function/F12022/list_global_best_fit.csv
Arquivo salvo: results/optimize-function/F12022_list_global_best_fit.csv
Arquivo não encontrado: ICA/optimize-function/F12022/list_exploitation.csv
Arquivo salvo: results/optimize-function/F12022_list_exploitation.csv
Arquivo não encontrado: ICA/optimize-function/F12022/list_exploration.csv
Arquivo salvo: results/optimize-function/F12022_list_exploration.csv
Arquivo não encontrado: ICA/optimize-function/F12022/list_current_best_fit.csv
Arquivo salvo: results/optimize-function/F12022_list_current_best_fit.csv
Arquivo não encontrado: ICA/optimize-function/F12022/list_diversity.csv
Arquivo salvo: results/optimize-function/F12022_list_diversity.csv
Arquivo não encontrado: ICA/optimize-function/F12022/list_epoch_time.csv
Arquivo salvo: results/optimize-function/F12022_list_epoch_time.csv
Arquivo não encontrado: ICA/optimize-function/F22022/list_global_best_fit.csv
Arquivo salvo: results/optimize-function/F22022_li

### TSP

In [ ]:
epoch = 10000
pop_size = 50

paras_bro = {
    "epoch": epoch,
    "pop_size": pop_size,
    "threshold": 3,
}
paras_improved_bso = {
    "epoch": epoch,
    "pop_size": pop_size,
    "m_clusters": 5,
    "p1": 0.2,
    "p2": 0.8,
    "p3": 0.4,
    "p4": 0.5,
}
paras_bso = dict(paras_improved_bso)
paras_bso["slope"] = 20
paras_ca = {
    "epoch": epoch,
    "pop_size": pop_size,
    "accepted_rate": 0.15,
}
paras_chio = {
    "epoch": epoch,
    "pop_size": pop_size,
    "brr": 0.15,
    "max_age": 3
}
paras_fbio = {
    "epoch": epoch,
    "pop_size": pop_size,
}
paras_gska = {
    "epoch": epoch,
    "pop_size": pop_size,
    "pb": 0.1,
    "kf": 0.5,
    "kr": 0.9,
    "kg": 5,
}
paras_ica = {
    "epoch": epoch,
    "pop_size": pop_size,
    "empire_count": 5,
    "assimilation_coeff": 1.5,
    "revolution_prob": 0.05,
    "revolution_rate": 0.1,
    "revolution_step_size": 0.1,
    "zeta": 0.1,
}
paras_lco = {
    "epoch": epoch,
    "pop_size": pop_size,
    "r1": 2.35,
}

paras_qsa = {
    "epoch": epoch,
    "pop_size": pop_size,
}
paras_saro = {
    "epoch": epoch,
    "pop_size": pop_size,
    "se": 0.5,
    "mu": 15
}
paras_ssdo = {
    "epoch": epoch,
    "pop_size": pop_size,
}
paras_tlo = {
    "epoch": epoch,
    "pop_size": pop_size,
}

term_dict = {
  #  "max_epoch": 100,
  #  "max_time": 60,
    "max_fe": 3000
}

In [ ]:
# Lista de algoritmos de otimização da mealpy
algorithms = {
    "BRO": BRO.OriginalBRO(**paras_bro),
    "BSO": BSO.OriginalBSO(**paras_bso),
    "CA": CA.OriginalCA(**paras_ca),
    "CHIO": CHIO.OriginalCHIO(**paras_chio),
    "FBIO": FBIO.OriginalFBIO(**paras_fbio),
    "GSKA": GSKA.OriginalGSKA(**paras_gska),
    # "ICA": ICA.OriginalICA(**paras_ica),
    "LCO": LCO.OriginalLCO(**paras_lco),
    "QSA": QSA.OriginalQSA(**paras_qsa),
    "SARO": SARO.OriginalSARO(**paras_saro),
    "SSDO": SSDO.OriginalSSDO(**paras_ssdo),
    "TLO": TLO.OriginalTLO(**paras_tlo)
}

In [ ]:
import numpy as np
from mealpy import PermutationVar, WOA, Problem

# Define the positions of the cities
city_positions = np.array([[60, 200], [180, 200], [80, 180], [140, 180], [20, 160],
                           [100, 160], [200, 160], [140, 140], [40, 120], [100, 120],
                           [180, 100], [60, 80], [120, 80], [180, 60], [20, 40],
                           [100, 40], [200, 40], [20, 20], [60, 20], [160, 20]])
num_cities = len(city_positions)
data = {
    "city_positions": city_positions,
    "num_cities": num_cities,
}

class TspProblem(Problem):
    def __init__(self, bounds=None, minmax="min", data=None, **kwargs):
        self.data = data
        super().__init__(bounds, minmax, **kwargs)

    @staticmethod
    def calculate_distance(city_a, city_b):
        # Calculate Euclidean distance between two cities
        return np.linalg.norm(city_a - city_b)

    @staticmethod
    def calculate_total_distance(route, city_positions):
        # Calculate total distance of a route
        total_distance = 0
        num_cities = len(route)
        for idx in range(num_cities):
            current_city = route[idx]
            next_city = route[(idx + 1) % num_cities]  # Wrap around to the first city
            total_distance += TspProblem.calculate_distance(city_positions[current_city], city_positions[next_city])
        return total_distance

    def obj_func(self, x):
        x_decoded = self.decode_solution(x)
        route = x_decoded["per_var"]
        fitness = self.calculate_total_distance(route, self.data["city_positions"])
        return fitness


bounds = PermutationVar(valid_set=list(range(0, num_cities)), name="per_var")
problem = TspProblem(bounds=bounds, minmax="min", data=data)

In [ ]:
for algo_name, algo in algorithms.items():
      print(f"\n\n==== Algorithm: {algo_name} =====")
      model = algo
      model.solve(problem, termination=term_dict)
      print(f"Best agent: {model.g_best}")
      print(f"Best solution: {model.g_best.solution}")
      print(f"Best accuracy: {model.g_best.target.fitness}")
      print(f"Best parameters: {model.problem.decode_solution(model.g_best.solution)}")
      model.history.save_global_objectives_chart(filename=f"{algo_name}/tsp/goc")
      model.history.save_local_objectives_chart(filename=f"{algo_name}/tsp/loc")
      model.history.save_global_best_fitness_chart(filename=f"{algo_name}/tsp/gbfc")
      model.history.save_local_best_fitness_chart(filename=f"{algo_name}/tsp/lbfc")
      model.history.save_runtime_chart(filename=f"{algo_name}/tsp/rtc")
      model.history.save_exploration_exploitation_chart(filename=f"{algo_name}/tsp/eec")
      model.history.save_diversity_chart(filename=f"{algo_name}/tsp/dc")
      save_history_to_csv(model, f"{algo_name}/tsp")

INFO:mealpy.human_based.BRO.OriginalBRO:Solving single objective optimization problem.
INFO:mealpy.human_based.BRO.OriginalBRO:>>>Problem: P, Epoch: 1, Current best: 1815.2230600509365, Global best: 1815.2230600509365, Runtime: 0.01850 seconds
INFO:mealpy.human_based.BRO.OriginalBRO:>>>Problem: P, Epoch: 2, Current best: 1751.2028093717574, Global best: 1751.2028093717574, Runtime: 0.01886 seconds
INFO:mealpy.human_based.BRO.OriginalBRO:>>>Problem: P, Epoch: 3, Current best: 1751.2028093717574, Global best: 1751.2028093717574, Runtime: 0.01702 seconds
INFO:mealpy.human_based.BRO.OriginalBRO:>>>Problem: P, Epoch: 4, Current best: 1751.2028093717574, Global best: 1751.2028093717574, Runtime: 0.01692 seconds
INFO:mealpy.human_based.BRO.OriginalBRO:>>>Problem: P, Epoch: 5, Current best: 1723.9412898970656, Global best: 1723.9412898970656, Runtime: 0.01626 seconds
INFO:mealpy.human_based.BRO.OriginalBRO:>>>Problem: P, Epoch: 6, Current best: 1723.9412898970656, Global best: 1723.94128989706



==== Algorithm: BRO =====


INFO:mealpy.human_based.BRO.OriginalBRO:>>>Problem: P, Epoch: 10, Current best: 1723.9412898970656, Global best: 1723.9412898970656, Runtime: 0.02048 seconds
INFO:mealpy.human_based.BRO.OriginalBRO:>>>Problem: P, Epoch: 11, Current best: 1723.9412898970656, Global best: 1723.9412898970656, Runtime: 0.01682 seconds
INFO:mealpy.human_based.BRO.OriginalBRO:>>>Problem: P, Epoch: 12, Current best: 1723.9412898970656, Global best: 1723.9412898970656, Runtime: 0.01688 seconds
INFO:mealpy.human_based.BRO.OriginalBRO:>>>Problem: P, Epoch: 13, Current best: 1723.9412898970656, Global best: 1723.9412898970656, Runtime: 0.01895 seconds
INFO:mealpy.human_based.BRO.OriginalBRO:>>>Problem: P, Epoch: 14, Current best: 1723.9412898970656, Global best: 1723.9412898970656, Runtime: 0.01688 seconds
INFO:mealpy.human_based.BRO.OriginalBRO:>>>Problem: P, Epoch: 15, Current best: 1723.9412898970656, Global best: 1723.9412898970656, Runtime: 0.01494 seconds
INFO:mealpy.human_based.BRO.OriginalBRO:>>>Problem: 

Best agent: id: 1095569, target: Objectives: [1488.14554204], Fitness: 1488.1455420410814, solution: [ 0  4  2  3  1  6  5 19 15  8 12 18 14 10 17 13 11 16  7  9]
Best solution: [ 0  4  2  3  1  6  5 19 15  8 12 18 14 10 17 13 11 16  7  9]
Best accuracy: 1488.1455420410814
Best parameters: {'per_var': [0, 4, 2, 3, 1, 6, 5, 18, 9, 19, 13, 16, 10, 15, 12, 8, 17, 14, 11, 7]}


INFO:mealpy.human_based.BSO.OriginalBSO:Solving single objective optimization problem.
INFO:mealpy.human_based.BSO.OriginalBSO:>>>Problem: P, Epoch: 1, Current best: 1822.5865525701286, Global best: 1822.5865525701286, Runtime: 0.03203 seconds
INFO:mealpy.human_based.BSO.OriginalBSO:>>>Problem: P, Epoch: 2, Current best: 1757.5350884917216, Global best: 1757.5350884917216, Runtime: 0.05922 seconds


...

Best agent: id: 1150770, target: Objectives: [1519.23410696], Fitness: 1519.2341069593253, solution: [14 13  9 15 19 12 17 16 18 10  5  8 11  4  7  1  3  6  0  2]
Best solution: [14 13  9 15 19 12 17 16 18 10  5  8 11  4  7  1  3  6  0  2]
Best accuracy: 1519.2341069593253
Best parameters: {'per_var': [18, 15, 19, 16, 13, 10, 17, 14, 11, 2, 9, 12, 5, 1, 0, 3, 7, 6, 8, 4]}


INFO:mealpy.human_based.TLO.OriginalTLO:Solving single objective optimization problem.
INFO:mealpy.human_based.TLO.OriginalTLO:>>>Problem: P, Epoch: 1, Current best: 1909.093238286895, Global best: 1909.093238286895, Runtime: 0.06009 seconds
INFO:mealpy.human_based.TLO.OriginalTLO:>>>Problem: P, Epoch: 2, Current best: 1896.3542816214951, Global best: 1896.3542816214951, Runtime: 0.06115 seconds




==== Algorithm: TLO =====


INFO:mealpy.human_based.TLO.OriginalTLO:>>>Problem: P, Epoch: 3, Current best: 1858.6328404716646, Global best: 1858.6328404716646, Runtime: 0.06537 seconds
INFO:mealpy.human_based.TLO.OriginalTLO:>>>Problem: P, Epoch: 4, Current best: 1817.3487656063726, Global best: 1817.3487656063726, Runtime: 0.06143 seconds
INFO:mealpy.human_based.TLO.OriginalTLO:>>>Problem: P, Epoch: 5, Current best: 1781.0432491651613, Global best: 1781.0432491651613, Runtime: 0.05968 seconds
INFO:mealpy.human_based.TLO.OriginalTLO:>>>Problem: P, Epoch: 6, Current best: 1777.2141309339452, Global best: 1777.2141309339452, Runtime: 0.03488 seconds
INFO:mealpy.human_based.TLO.OriginalTLO:>>>Problem: P, Epoch: 7, Current best: 1777.2141309339452, Global best: 1777.2141309339452, Runtime: 0.03995 seconds
INFO:mealpy.human_based.TLO.OriginalTLO:>>>Problem: P, Epoch: 8, Current best: 1748.3977909734758, Global best: 1748.3977909734758, Runtime: 0.03614 seconds
INFO:mealpy.human_based.TLO.OriginalTLO:>>>Problem: P, Epo

Best agent: id: 1153886, target: Objectives: [1522.18501143], Fitness: 1522.1850114266883, solution: [ 3  1  4  0  6  5  2  8  7 10  9 13 11 16 12 14 17 15 19 18]
Best solution: [ 3  1  4  0  6  5  2  8  7 10  9 13 11 16 12 14 17 15 19 18]
Best accuracy: 1522.1850114266883
Best parameters: {'per_var': [3, 1, 6, 0, 2, 5, 4, 8, 7, 10, 9, 12, 14, 11, 15, 17, 13, 16, 19, 18]}


In [ ]:
# Funções e algoritmos
algorithms = [ "BRO", "BSO", "CA", "CHIO", "FBIO", "GSKA", "ICA", "LCO", "QSA", "SARO", "SSDO", "TLO"]
metrics = ["list_global_best_fit", "list_exploitation", "list_exploration", "list_current_best_fit", "list_diversity", "list_epoch_time"]

# Caminho base dos dados
base_path = ""  # Substitua pelo caminho real dos dados


# Função para carregar os dados e consolidar em um DataFrame
def consolidate_data(metric):
    consolidated_df = pd.DataFrame()
    for alg in algorithms:
        file_path = os.path.join(base_path, f"{alg}/tsp/{metric}.csv")
        if os.path.exists(file_path):
            df = pd.read_csv(file_path)
            df.columns = [f"{alg}_{col}" for col in df.columns]
            if consolidated_df.empty:
                consolidated_df = df
            else:
                consolidated_df = pd.concat([consolidated_df, df], axis=1)
        else:
            print(f"Arquivo não encontrado: {file_path}")
    return consolidated_df

# Processar cada métrica
for metric in metrics:
    consolidated_df = consolidate_data(metric)
    output_file = f"results/tsp/{metric}_tsp.csv"
    consolidated_df.to_csv(output_file, index=False)
    print(f"Arquivo salvo: {output_file}")

Arquivo não encontrado: ICA/tsp/list_global_best_fit.csv
Arquivo salvo: results/tsp/list_global_best_fit_tsp.csv
Arquivo não encontrado: ICA/tsp/list_exploitation.csv
Arquivo salvo: results/tsp/list_exploitation_tsp.csv
Arquivo não encontrado: ICA/tsp/list_exploration.csv
Arquivo salvo: results/tsp/list_exploration_tsp.csv
Arquivo não encontrado: ICA/tsp/list_current_best_fit.csv
Arquivo salvo: results/tsp/list_current_best_fit_tsp.csv
Arquivo não encontrado: ICA/tsp/list_diversity.csv
Arquivo salvo: results/tsp/list_diversity_tsp.csv
Arquivo não encontrado: ICA/tsp/list_epoch_time.csv
Arquivo salvo: results/tsp/list_epoch_time_tsp.csv
